# Add Model Tutorial
This notebook give a tutorial on adding a base model that supports benchmark tasks.

### Model API and Considerations
1. The model needs to implement the interface `forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec)`, which is forward propagation, and the meaning of each parameter is as follows:
   * input:
     * `x_enc`: the input encoder sequence data
     * `x_mark_enc`: marks associated with the encoder sequence data, such as timestamps, category labels, etc.
     * `x_dec`: Input sequence data of the decoder
     * `x_mark_dec`: decoder sequence data related markers
   * output:
     * the return is a 3D tensor with dimensions [batch_size, pred_len, feature_num].
2. model initialization (\_\_init\_\_), accept related configuration parameters, configuration parameters can be used in the model, see example for details.
3. model class must inherit from `torch.nn.Module`, otherwise it cannot be accessed by the optimizer.
4. In the \_\_init\_\_ function of the `exp/exp_basic.py` file, add your model name in the dictionary, so that the framework reads your model.

One modeling framework is roughly:

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, configs):
        super(MyModel, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        # other configurations
        # model initialization
        # ......
        
    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        # encoder and decoder
        # ......
        pass
        # return dec_out
        
    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        # depending on the task name, you can choose different ways of forward propagation, here is the prediction task
        dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
        return dec_out[:, -self.pred_len:, :]  # [B, L, D]

### Using the iTransformer model as a proven example.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from layers.Transformer_EncDec import Encoder, EncoderLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
from layers.Embed import DataEmbedding_inverted
import numpy as np


class Model(nn.Module):
    """
    Paper link: https://arxiv.org/abs/2310.06625
    """

    def __init__(self, configs):
        super(Model, self).__init__()
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        # Embedding
        self.enc_embedding = DataEmbedding_inverted(configs.seq_len, configs.d_model, configs.embed, configs.freq,
                                                    configs.dropout)
        # Encoder
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        FullAttention(False, configs.factor, attention_dropout=configs.dropout,
                                      output_attention=False), configs.d_model, configs.n_heads),
                    configs.d_model,
                    configs.d_ff,
                    dropout=configs.dropout,
                    activation=configs.activation
                ) for l in range(configs.e_layers)
            ],
            norm_layer=torch.nn.LayerNorm(configs.d_model)
        )
        # Decoder
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.projection = nn.Linear(configs.d_model, configs.pred_len, bias=True)

    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        # Normalization from Non-stationary Transformer
        means = x_enc.mean(1, keepdim=True).detach()
        x_enc = x_enc - means
        stdev = torch.sqrt(torch.var(x_enc, dim=1, keepdim=True, unbiased=False) + 1e-5)
        x_enc /= stdev

        _, _, N = x_enc.shape

        # Embedding
        enc_out = self.enc_embedding(x_enc, x_mark_enc)
        enc_out, attns = self.encoder(enc_out, attn_mask=None)

        dec_out = self.projection(enc_out).permute(0, 2, 1)[:, :, :N]
        # De-Normalization from Non-stationary Transformer
        dec_out = dec_out * (stdev[:, 0, :].unsqueeze(1).repeat(1, self.pred_len, 1))
        dec_out = dec_out + (means[:, 0, :].unsqueeze(1).repeat(1, self.pred_len, 1))
        return dec_out

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc, x_mark_enc, x_dec, x_mark_dec)
            return dec_out[:, -self.pred_len:, :]  # [B, L, D]
        return None